In [36]:
a = rand(10^7)

10000000-element Array{Float64,1}:
 0.006816440332839679
 0.2604644917932668
 0.7200281335755259
 0.5500287187349284
 0.5349505780202581
 0.8108511556217324
 0.6518275810969822
 0.15770434641090691
 0.4292120124769352
 0.3908141394207365
 0.31046025683075507
 0.7581011249834781
 0.013416029606780677
 ⋮
 0.4006628539168444
 0.8542479437476522
 0.8084827857238435
 0.03323038240330156
 0.6331276126300021
 0.18027617637294946
 0.8463791567842283
 0.38913549514328816
 0.5002356473423357
 0.2241089229114248
 0.41887176547106963
 0.17655650855268434

In [37]:
sum(a)

4.999434582630457e6

In [38]:
#using Pkg
#Pkg.add("BenchmarkTools")
using BenchmarkTools

In [39]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

"#include <stddef.h>\ndouble c_sum(size_t n, double *X) {\n    double s = 0.0;\n    for (size_t i = 0; i < n; ++i) {\n        s += X[i];\n    }\n    return s;\n}\n"

In [40]:
const Clib = tempname()
println(Clib)

/tmp/jl_B3DnzG


In [41]:
using Libdl

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [42]:
c_sum(a)

4.9994345826301165e6

In [43]:
c_sum(a) ≈ sum(a)

true

In [44]:
methods(≈)

# 9 methods for generic function "isapprox":
[1] isapprox(::Missing, ::Missing; kwargs...) in Base at missing.jl:89
[2] isapprox(::Missing, ::Any; kwargs...) in Base at missing.jl:90
[3] isapprox(x::AbstractArray, y::AbstractArray; atol, rtol, nans, norm) in LinearAlgebra at /home/neo/packages/julias/julia-1.5/share/julia/stdlib/v1.5/LinearAlgebra/src/generic.jl:1631
[4] isapprox(J1::LinearAlgebra.UniformScaling{T}, J2::LinearAlgebra.UniformScaling{S}; atol, rtol, nans) where {T<:Number, S<:Number} in LinearAlgebra at /home/neo/packages/julias/julia-1.5/share/julia/stdlib/v1.5/LinearAlgebra/src/uniformscaling.jl:292
[5] isapprox(J::LinearAlgebra.UniformScaling, A::AbstractArray{T,2} where T; atol, rtol, nans, norm) in LinearAlgebra at /home/neo/packages/julias/julia-1.5/share/julia/stdlib/v1.5/LinearAlgebra/src/uniformscaling.jl:296
[6] isapprox(A::AbstractArray{T,2} where T, J::LinearAlgebra.UniformScaling; kwargs...) in LinearAlgebra at /home/neo/packages/julias/julia-1.5/share/julia/stdlib/v1.5/LinearAlgebra/src/uniformscaling.jl:306
[7] isapprox(y; kwargs...) in Base at floatfuncs.jl:287
[8] isapprox(::Any, ::Missing; kwargs...) in Base at missing.jl:91
[9] isapprox(x::Number, y::Number; atol, rtol, nans) in Base at floatfuncs.jl:276

In [45]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=atol>0 ? 0 : √eps, atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significand digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to the usual `norm` function in LinearAlgebra, but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

# Examples

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true
```

---

```
isapprox(x; kwargs...) / ≈(x; kwargs...)
```

Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.

The keyword arguments supported here are the same as those in the 2-argument `isapprox`.


In [46]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.133 ms (0.00% GC)
  median time:      7.204 ms (0.00% GC)
  mean time:        7.210 ms (0.00% GC)
  maximum time:     7.432 ms (0.00% GC)
  --------------
  samples:          694
  evals/sample:     1

In [47]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 7.13338 msec


In [48]:
d = Dict()
d["C"] = minimum(c_bench.times) / 1e6 # in millisecods
d

Dict{Any,Any} with 1 entry:
  "C" => 7.13338

In [49]:
#Pkg.add("PyCall")
using PyCall

In [50]:
pysum = pybuiltin("sum")

pysum(a)

pysum(a) ≈ sum(a)

true

In [51]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     1.454 s (0.00% GC)
  median time:      1.475 s (0.00% GC)
  mean time:        1.476 s (0.00% GC)
  maximum time:     1.498 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [52]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 7.13338
  "Python built-in" => 1454.46

In [53]:
#Pkg.add("Conda")
using Conda
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda /home/neo/.julia/packages/Conda/x5ml4/src/Conda.jl:115


Solving environment: ...working... done

# All requested packages already installed.



In [57]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a)
py_numpy_bench = @benchmark $numpy_sum($apy_numpy)
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6

3.228927

In [55]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.813 ms (0.00% GC)
  median time:      3.002 ms (0.00% GC)
  mean time:        3.053 ms (0.00% GC)
  maximum time:     4.433 ms (0.00% GC)
  --------------
  samples:          1637
  evals/sample:     1

In [58]:
d["Julia build-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"               => 7.13338
  "Python numpy"    => 3.22893
  "Python built-in" => 1454.46
  "Julia build-in"  => 2.81335

In [65]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [67]:
mysum(a) ≈ sum(a)

true

In [68]:
my_bench = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.250 ms (0.00% GC)
  median time:      7.377 ms (0.00% GC)
  mean time:        7.407 ms (0.00% GC)
  maximum time:     8.104 ms (0.00% GC)
  --------------
  samples:          675
  evals/sample:     1

In [69]:
d["Julia self written"] = minimum(my_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                  => 7.13338
  "Python numpy"       => 3.22893
  "Python built-in"    => 1454.46
  "Julia self written" => 7.24999
  "Julia build-in"     => 2.81335